
#### Description of QueryProcessor


##### Features

- Dynamic table_changes for tables with version_param.
- Optional per-table filter applied automatically.
- Multi-column joins handled correctly.
- Union across multiple tables.
- Dynamic catalog replacements like ${entity} supported.
-  Runtime version map allows flexible filtering of selected tables.

---

##### 1. parseQueryFromYaml

This function can be used to check the the generated SQL query before actual test, we can copy the query to SQL editor to run to validate first 

```scala
def parseQueryFromYaml(yamlSql: String, catalogMap: Map[String, String]): String = { ...  }
```

Paramters:
- yamlSql   The SQL-query configuration that defines the SQL query. It contains placeholders(catalog) that need to be replaced.
- catalogMap:  A map of placeholder → actual value. the map data can be created using SchemaResolver 

```scala
val catalogMap = SchemaResolver.getAcsSchemaMap()
```

Example

```scala
 Map(
  "entity" -> "ag_content_ims_acs_prod.gold_entity",
  "wos" -> "ag_content_ims_acs_prod.gold_wos",
  "pprn" -> "ag_content_ims_acs_prod.gold_pprn",
  "dap" -> "ag_ra_search_analytics_data_dev.sandbox_v1_0"
)
```

---


##### 2. runUnionQueryAndSave

This function  is used to actually create the delta  table speficied in SQL config and  save the affected PK value to the table

```scala
def runUnionQueryAndSave(yamlSql: String, versionMap: Map[String, (Long, Long)] = Map.empty): Unit = { ... }
```

Paramters:

- yamlSql:    The SQL-query configuration that defines the SQL query. It contains placeholders(catalog) that need to be replaced.  
- versionMap:   A map from upstream table name → (startVersion, endVersion), where `startVersion` and `endVersion` define the inclusive version range to process for each upstream table.  the map data can be created using WatermarkManager 


---





#### 1. Setup Param 

In [0]:
//  pass the parameters
dbutils.widgets.text("source_catalog", "ag_content_ims_acs")
dbutils.widgets.text("source_environment", "prod")
dbutils.widgets.text("source_version", "v1_0_0")

dbutils.widgets.text("target_catalog", "ag_ra_search_analytics_data")
dbutils.widgets.text("target_environment", "dev")
dbutils.widgets.text("target_version", "v1_0")

// dynamic paramters
val source_catalog = dbutils.widgets.get("source_catalog")
val source_environment = dbutils.widgets.get("source_environment")
val source_version = dbutils.widgets.get("source_version")
val target_catalog = dbutils.widgets.get("target_catalog")
val target_environment = dbutils.widgets.get("target_environment")
val target_version = dbutils.widgets.get("target_version")


source_catalog: String = "ag_content_ims_acs"
source_environment: String = "prod"
source_version: String = ""
target_catalog: String = "ag_ra_search_analytics_data"
target_environment: String = "dev"
target_version: String = "v1_0"

#### 2. QueryProcessor

In [0]:

object SchemaResolver {

  private def getWidget(name: String, default: String): String = {
    try {
      val value = dbutils.widgets.get(name)
      if (value == null || value.isEmpty) default else value
    } catch {
      case _: Throwable => default
    }
  }

  private val source_catalog = getWidget("source_catalog", "ag_content_ims_acs")
  private val source_environment = getWidget("source_environment", "prod")
  private val source_version = getWidget("source_version", "")

  private val target_catalog = getWidget("target_catalog", "ag_ra_search_analytics_data")
  private val target_environment = getWidget("target_environment", "dev")
  private val target_version = getWidget("target_version", "v1_0")

  
  def getAcsSchemaMap():  Map[String, String]  = {
     val version = if(source_version.isEmpty) "" else  s"_${source_version}"

     Map(
      "entity" -> s"${source_catalog}_${source_environment}.gold_entity${version}",
      "wos" -> s"${source_catalog}_${source_environment}.gold_wos${version}",
      "pprn" -> s"${source_catalog}_${source_environment}.gold_pprn${version}",
      "dap" -> s"${target_catalog}_${target_environment}.sandbox_${target_version}",
    )
  }


  def getAcsSchemas():  Seq[String]  = {
     val version = if(source_version.isEmpty) "" else  s"_${source_version}"

     Seq(
       s"${source_catalog}_${source_environment}.gold_entity${version}",
       s"${source_catalog}_${source_environment}.gold_wos${version}",
       s"${source_catalog}_${source_environment}.gold_pprn${version}"
    )
   
  }
}


val catalogMap = SchemaResolver.getAcsSchemaMap()

println(catalogMap)

Map(entity -> ag_content_ims_acs_prod.gold_entity, wos -> ag_content_ims_acs_prod.gold_wos, pprn -> ag_content_ims_acs_prod.gold_pprn, dap -> ag_ra_search_analytics_data_dev.sandbox_v1_0)


defined object SchemaResolver
replacements: Map[String, String] = Map(
  "entity" -> "ag_content_ims_acs_prod.gold_entity",
  "wos" -> "ag_content_ims_acs_prod.gold_wos",
  "pprn" -> "ag_content_ims_acs_prod.gold_pprn",
  "dap" -> "ag_ra_search_analytics_data_dev.sandbox_v1_0"
)

In [0]:
import org.yaml.snakeyaml.Yaml
import scala.jdk.CollectionConverters._
import org.apache.spark.sql.DataFrame

// ---------------------- Case Classes ----------------------
case class TableJoin(
  table_name: String,
  alias: String,
  pk: String,
  fk: Option[String] = None,
  versionParam: Option[String] = None,  // dynamic version key
  filter: Option[String] = None         // optional WHERE filter
)

case class UnionTable(
  select_columns: String,
  joins: Seq[TableJoin]
)

case class OutputTable(name: String)

case class UnionConfig(
  output_table: OutputTable,
  union_tables: Seq[UnionTable]
)

object QueryProcessor {

  private def cleanTableName(name: String): String =
    name.replace("\"", "").trim

  // ------------------ YAML Parser ------------------
  def parseQueryFromYaml(yamlSql: String, catalogMap: Map[String, String]): UnionConfig = {
    var replacedYaml = yamlSql
    catalogMap.foreach { case (key, value) =>
      replacedYaml = replacedYaml.replace(s"$${$key}", value)
    }

    val yaml = new Yaml()
    val obj = yaml.load(replacedYaml).asInstanceOf[java.util.Map[String, Object]]

    val outputTableMap = obj.get("output_table").asInstanceOf[java.util.Map[String, Object]]
    val outputTable = OutputTable(outputTableMap.get("name").toString)

    val unionTablesJava = obj.get("union_tables")
      .asInstanceOf[java.util.List[java.util.Map[String, Object]]]

    val unionTables = unionTablesJava.asScala.map { ut =>
      val joinsJava = ut.get("joins")
        .asInstanceOf[java.util.List[java.util.Map[String, Object]]]

      UnionTable(
        select_columns = ut.get("select_columns").toString,
        joins = joinsJava.asScala.map { j =>
          TableJoin(
            table_name = cleanTableName(j.get("table_name").toString),
            alias = j.get("alias").toString,
            pk = j.get("pk").toString,
            fk = Option(j.get("fk")).map(_.toString),
            versionParam = Option(j.get("version_param")).map(_.toString),
            filter = Option(j.get("filter")).map(_.toString)
          )
        }.toSeq
      )
    }

    UnionConfig(outputTable, unionTables.toSeq)
  }

  // ------------------ Query Generator ------------------
  def generateJoinQuery(
      tables: Seq[TableJoin],
      selectColumns: String,
      versionMap: Map[String, (Long, Long)]
  ): String = {

    val base = tables.head

    def tableRef(t: TableJoin): String = {
      t.versionParam.flatMap(versionMap.get) match {
        case Some((start, end)) =>
          s"table_changes('${t.table_name}', $start, $end) ${t.alias}"
        case None =>
          s"${t.table_name} ${t.alias}"
      }
    }

    val joins = tables.tail.map { t =>
      val pkCols = t.pk.split(",").map(_.trim)
      val fkCols = t.fk.map(_.split(",").map(_.trim)).getOrElse(Array.empty)

      val cond =
        if (fkCols.isEmpty) ""
        else pkCols.zip(fkCols).map { case (pk, fk) => s"$pk = $fk" }.mkString(" AND ")

      s"JOIN ${tableRef(t)} ON $cond"
    }.mkString("\n")

    // Filter must apply AFTER joins
    val where =
      base.filter.map(f => s"WHERE $f").getOrElse("")

    s"""
       |SELECT $selectColumns
       |FROM ${tableRef(base)}
       |$joins
       |$where
       |""".stripMargin.trim
  }

  // ------------------ Union Query ------------------
  def generateUnionQueryFromConfig(
      config: UnionConfig,
      versionMap: Map[String, (Long, Long)]
  ): String =
    config.union_tables.map { ut =>
      generateJoinQuery(ut.joins, ut.select_columns, versionMap)
    }.mkString("\nUNION\n")


  // ------------------ Run Query and Display ------------------
  def runUnionQueryAndSave(
      yamlConfig: String,
      versionMap: Map[String, (Long, Long)] = Map.empty,
      dryRun:Boolean = true
  )(implicit spark: org.apache.spark.sql.SparkSession): Unit = {

    val catalogMap = SchemaResolver.getAcsSchemaMap()
    val config = parseQueryFromYaml(yamlConfig, catalogMap)
    val sqlQuery = generateUnionQueryFromConfig(config, versionMap)

    println("Generated SQL Query:\n" + sqlQuery)

    val resultDf: DataFrame = spark.sql(sqlQuery)
    resultDf.show()  // or display(resultDf) in Databricks

    // Save as Delta table if needed
    resultDf.write.format("delta").mode("overwrite").saveAsTable(config.output_table.name)

    println(s"Saved result to Delta table: ${config.output_table.name}")
  }

}


import org.yaml.snakeyaml.Yaml

import scala.jdk.CollectionConverters._

import org.apache.spark.sql.DataFrame

// ---------------------- Case Classes ----------------------

defined class TableJoin
defined class UnionTable
defined class OutputTable
defined class UnionConfig
defined object QueryProcessor

#### 3. Test QueryProcessor - without version

In [0]:
val queryAlma: String =  """
output_table:
  name: "${dap}.pk_alma_affected"

union_tables:
  - select_columns: "journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"

  - select_columns: "journal_key AS journal_key"
    joins:
      - table_name: "${entity}.d_alma_subscriptions"
        alias: "alma"
        pk: "journal_key"

  - select_columns: "jlink.journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.publisher_publication_link"
        alias: "plink"
        pk: "uid"
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        fk: "plink.uid = jlink.uid"

"""

queryAlma: String = """
output_table:
  name: "${dap}.pk_alma_affected"

union_tables:
  - select_columns: "journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"

  - select_columns: "journal_key AS journal_key"
    joins:
      - table_name: "${entity}.d_alma_subscriptions"
        alias: "alma"
        pk: "journal_key"

  - select_columns: "jlink.journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.publisher_publication_link"
        alias: "plink"
        pk: "uid"
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        fk: "plink.uid = jlink.uid"

"""

In [0]:

val config = QueryProcessor.parseQueryFromYaml(queryAlma, catalogMap)
val sqlQuery = QueryProcessor.generateUnionQueryFromConfig(config, null)

println(sqlQuery.stripMargin)

SELECT DISTINCT journal_acs_key AS journal_key
FROM ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link jlink
UNION
SELECT DISTINCT journal_key AS journal_key
FROM ag_content_ims_acs_prod.gold_entity.d_alma_subscriptions alma
UNION
SELECT DISTINCT jlink.journal_acs_key AS journal_key
FROM ag_content_ims_acs_prod.gold_wos.publisher_publication_link plink
JOIN ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link jlink ON journal_acs_key = plink.uid = jlink.uid


config: UnionConfig = UnionConfig(
  output_table = OutputTable(
    name = "ag_ra_search_analytics_data_dev.sandbox_v1_0.pk_alma_affected"
  ),
  union_tables = List(
    UnionTable(
      select_columns = "journal_acs_key AS journal_key",
      joins = List(
        TableJoin(
          table_name = "ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link",
          alias = "jlink",
          pk = "journal_acs_key",
          fk = None,
          versionParam = None,
          filter = None
        )
      )
    ),
    UnionTable(
      select_columns = "journal_key AS journal_key",
      joins = List(
        TableJoin(
          table_name = "ag_content_ims_acs_prod.gold_entity.d_alma_subscriptions",
          alias = "alma",
          pk = "journal_key",
          fk = None,
          versionParam = None,
          filter = None
        )
      )
    ),
    UnionTable(
      select_columns = "jlink.journal_acs_key AS journal_key",
      joins = List(
        TableJoin(
    

#### 4. Test QueryProcessor- with version

In [0]:
val queryAP: String =  """
output_table:
  name: "${dap}.pk_affected_sp"

union_tables:
  - select_columns: "sp_id AS sp_id, sp._change_type as _change_type"
    joins:
      - table_name: ${entity}.d_spmaster
        alias: "sp"
        pk: "sp_id"
        version_param: "d_spmaster"   # optional: will use dynamic version
        filter: "sp._change_type IN ('insert', 'delete', 'update_postimage')"  # optional

  - select_columns: "spl.sp_id AS sp_id,  'update_postimage' AS _change_type"
    joins:
      # Base table: updated orgmaster
      - table_name: ${entity}.d_orgmaster
        alias: org
        pk: "org.org_pguid"
        version_param: d_orgmaster 
        filter: "org._change_type IN ('insert', 'delete', 'update_postimage')"

      # Join to orgmaster_publication_link (by org_pguid)
      - table_name: ${entity}.orgmaster_publication_link
        alias: oml
        pk: "oml.org_pguid"
        fk: "org.org_pguid"

      # Join to spmaster_publication_link (by uid AND author_position)
      - table_name: ${entity}.spmaster_publication_link
        alias: spl
        pk: "spl.uid, spl.author_position"
        fk: "oml.uid, oml.address_position"


"""

queryAP: String = """
output_table:
  name: "${dap}.pk_affected_sp"

union_tables:
  - select_columns: "sp_id AS sp_id, sp._change_type as _change_type"
    joins:
      - table_name: ${entity}.d_spmaster
        alias: "sp"
        pk: "sp_id"
        version_param: "d_spmaster"   # optional: will use dynamic version
        filter: "sp._change_type IN ('insert', 'delete', 'update_postimage')"  # optional

  - select_columns: "spl.sp_id AS sp_id,  'update_postimage' AS _change_type"
    joins:
      # Base table: updated orgmaster
      - table_name: ${entity}.d_orgmaster
        alias: org
        pk: "org.org_pguid"
        version_param: d_orgmaster 
        filter: "org._change_type IN ('insert', 'delete', 'update_postimage')"

      # Join to orgmaster_publication_link (by org_pguid)
      - table_name: ${entity}.orgmaster_publication_link
        alias: oml
        pk: "oml.org_pguid"
        fk: "org.org_pguid"

      # Join to spmaster_publication_link (by uid AND author_posit

In [0]:

val versionMap = Map(
  "d_spmaster" -> (1L, 2L),
  "d_orgmaster" -> (1L, 2L)
)

val config = QueryProcessor.parseQueryFromYaml(queryAP, catalogMap)
val sqlQuery = QueryProcessor.generateUnionQueryFromConfig(config, versionMap)

println(sqlQuery.stripMargin)



SELECT sp_id AS sp_id, sp._change_type as _change_type
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_spmaster', 1, 2) sp

WHERE sp._change_type IN ('insert', 'delete', 'update_postimage')
UNION
SELECT spl.sp_id AS sp_id,  'update_postimage' AS _change_type
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 2) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE org._change_type IN ('insert', 'delete', 'update_postimage')


versionMap: Map[String, (Long, Long)] = Map(
  "d_spmaster" -> (1L, 2L),
  "d_orgmaster" -> (1L, 2L)
)
config: UnionConfig = UnionConfig(
  output_table = OutputTable(
    name = "ag_ra_search_analytics_data_dev.sandbox_v1_0.pk_affected_sp"
  ),
  union_tables = List(
    UnionTable(
      select_columns = "sp_id AS sp_id, sp._change_type as _change_type",
      joins = List(
        TableJoin(
          table_name = "ag_content_ims_acs_prod.gold_entity.d_spmaster",
          alias = "sp",
          pk = "sp_id",
          fk = None,
          versionParam = Some(value = "d_spmaster"),
          filter = Some(
            value = "sp._change_type IN ('insert', 'delete', 'update_postimage')"
          )
        )
      )
    ),
    UnionTable(
      select_columns = "spl.sp_id AS sp_id,  'update_postimage' AS _change_type",
      joins = List(
        TableJoin(
          table_name = "ag_content_ims_acs_prod.gold_entity.d_orgmaster",
          alias = "org",
          pk = "org.org_pgu

In [0]:

// dryRun - Create pk_affected table pk_affected_sp

QueryProcessor.runUnionQueryAndSave(queryAP, versionMap. true)



Generated SQL Query:
SELECT sp_id AS sp_id
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_spmaster', 1, 2) sp

WHERE sp._change_type IN ('insert', 'delete', 'update_postimage')
UNION
SELECT spl.sp_id AS sp_id
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 2) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE org._change_type IN ('insert', 'delete', 'update_postimage')
+----------------+
|           sp_id|
+----------------+
|urn:spm:10000216|
|urn:spm:10000029|
|urn:spm:10000104|
|urn:spm:10000135|
|urn:spm:10000134|
| urn:spm:1000015|
|urn:spm:10000046|
|urn:spm:10000072|
|urn:spm:10000008|
| urn:spm:1000011|
|urn:spm:10000176|
|urn:spm:10000026|
|urn:spm:10000097|
|urn:spm:10000037|
|urn:spm:10000041|
|   urn:spm:10000|
|urn:spm:10000180|
|urn:spm: